# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f474cd9f9d0>
├── 'a' --> tensor([[-0.4202, -0.0075,  0.3304],
│                   [-0.6290, -0.2106, -0.2328]])
└── 'x' --> <FastTreeValue 0x7f474cd9faf0>
    └── 'c' --> tensor([[-1.3071, -1.8390, -0.8650, -1.0303],
                        [ 0.7796,  1.2613, -0.3053, -0.5583],
                        [ 2.0123, -0.6229,  0.1774, -0.0540]])

In [4]:
t.a

tensor([[-0.4202, -0.0075,  0.3304],
        [-0.6290, -0.2106, -0.2328]])

In [5]:
%timeit t.a

69.6 ns ± 1.37 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f474cd9f9d0>
├── 'a' --> tensor([[ 0.4728, -0.7852, -1.2777],
│                   [ 1.2646, -0.7693,  0.4981]])
└── 'x' --> <FastTreeValue 0x7f474cd9faf0>
    └── 'c' --> tensor([[-1.3071, -1.8390, -0.8650, -1.0303],
                        [ 0.7796,  1.2613, -0.3053, -0.5583],
                        [ 2.0123, -0.6229,  0.1774, -0.0540]])

In [7]:
%timeit t.a = new_value

75.7 ns ± 0.585 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.4202, -0.0075,  0.3304],
               [-0.6290, -0.2106, -0.2328]]),
    x: Batch(
           c: tensor([[-1.3071, -1.8390, -0.8650, -1.0303],
                      [ 0.7796,  1.2613, -0.3053, -0.5583],
                      [ 2.0123, -0.6229,  0.1774, -0.0540]]),
       ),
)

In [10]:
b.a

tensor([[-0.4202, -0.0075,  0.3304],
        [-0.6290, -0.2106, -0.2328]])

In [11]:
%timeit b.a

64.3 ns ± 1.02 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.3577, -0.5311,  0.3426],
               [-1.1936, -1.7176, -1.0226]]),
    x: Batch(
           c: tensor([[-1.3071, -1.8390, -0.8650, -1.0303],
                      [ 0.7796,  1.2613, -0.3053, -0.5583],
                      [ 2.0123, -0.6229,  0.1774, -0.0540]]),
       ),
)

In [13]:
%timeit b.a = new_value

566 ns ± 14.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

956 ns ± 16.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.4 µs ± 163 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

265 µs ± 6.99 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

261 µs ± 11.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f46a2a2b940>
├── 'a' --> tensor([[[-0.4202, -0.0075,  0.3304],
│                    [-0.6290, -0.2106, -0.2328]],
│           
│                   [[-0.4202, -0.0075,  0.3304],
│                    [-0.6290, -0.2106, -0.2328]],
│           
│                   [[-0.4202, -0.0075,  0.3304],
│                    [-0.6290, -0.2106, -0.2328]],
│           
│                   [[-0.4202, -0.0075,  0.3304],
│                    [-0.6290, -0.2106, -0.2328]],
│           
│                   [[-0.4202, -0.0075,  0.3304],
│                    [-0.6290, -0.2106, -0.2328]],
│           
│                   [[-0.4202, -0.0075,  0.3304],
│                    [-0.6290, -0.2106, -0.2328]],
│           
│                   [[-0.4202, -0.0075,  0.3304],
│                    [-0.6290, -0.2106, -0.2328]],
│           
│                   [[-0.4202, -0.0075,  0.3304],
│                    [-0.6290, -0.2106, -0.2328]]])
└── 'x' --> <FastTreeValue 0x7f474cdcf4c0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

41.9 µs ± 585 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f46a2a2b2e0>
├── 'a' --> tensor([[-0.4202, -0.0075,  0.3304],
│                   [-0.6290, -0.2106, -0.2328],
│                   [-0.4202, -0.0075,  0.3304],
│                   [-0.6290, -0.2106, -0.2328],
│                   [-0.4202, -0.0075,  0.3304],
│                   [-0.6290, -0.2106, -0.2328],
│                   [-0.4202, -0.0075,  0.3304],
│                   [-0.6290, -0.2106, -0.2328],
│                   [-0.4202, -0.0075,  0.3304],
│                   [-0.6290, -0.2106, -0.2328],
│                   [-0.4202, -0.0075,  0.3304],
│                   [-0.6290, -0.2106, -0.2328],
│                   [-0.4202, -0.0075,  0.3304],
│                   [-0.6290, -0.2106, -0.2328],
│                   [-0.4202, -0.0075,  0.3304],
│                   [-0.6290, -0.2106, -0.2328]])
└── 'x' --> <FastTreeValue 0x7f46a2a2bc70>
    └── 'c' --> tensor([[-1.3071, -1.8390, -0.8650, -1.0303],
                        [ 0.7796,  1.2613, -0.3053, -0.5583],
                 

In [23]:
%timeit t_cat(trees)

39.7 µs ± 1.22 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

74.5 µs ± 1.62 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.4202, -0.0075,  0.3304],
                [-0.6290, -0.2106, -0.2328]],
       
               [[-0.4202, -0.0075,  0.3304],
                [-0.6290, -0.2106, -0.2328]],
       
               [[-0.4202, -0.0075,  0.3304],
                [-0.6290, -0.2106, -0.2328]],
       
               [[-0.4202, -0.0075,  0.3304],
                [-0.6290, -0.2106, -0.2328]],
       
               [[-0.4202, -0.0075,  0.3304],
                [-0.6290, -0.2106, -0.2328]],
       
               [[-0.4202, -0.0075,  0.3304],
                [-0.6290, -0.2106, -0.2328]],
       
               [[-0.4202, -0.0075,  0.3304],
                [-0.6290, -0.2106, -0.2328]],
       
               [[-0.4202, -0.0075,  0.3304],
                [-0.6290, -0.2106, -0.2328]]]),
    x: Batch(
           c: tensor([[[-1.3071, -1.8390, -0.8650, -1.0303],
                       [ 0.7796,  1.2613, -0.3053, -0.5583],
                       [ 2.0123, -0.6229,  0.1774, -0.0540]],
         

In [26]:
%timeit Batch.stack(batches)

96.7 µs ± 1.63 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.4202, -0.0075,  0.3304],
               [-0.6290, -0.2106, -0.2328],
               [-0.4202, -0.0075,  0.3304],
               [-0.6290, -0.2106, -0.2328],
               [-0.4202, -0.0075,  0.3304],
               [-0.6290, -0.2106, -0.2328],
               [-0.4202, -0.0075,  0.3304],
               [-0.6290, -0.2106, -0.2328],
               [-0.4202, -0.0075,  0.3304],
               [-0.6290, -0.2106, -0.2328],
               [-0.4202, -0.0075,  0.3304],
               [-0.6290, -0.2106, -0.2328],
               [-0.4202, -0.0075,  0.3304],
               [-0.6290, -0.2106, -0.2328],
               [-0.4202, -0.0075,  0.3304],
               [-0.6290, -0.2106, -0.2328]]),
    x: Batch(
           c: tensor([[-1.3071, -1.8390, -0.8650, -1.0303],
                      [ 0.7796,  1.2613, -0.3053, -0.5583],
                      [ 2.0123, -0.6229,  0.1774, -0.0540],
                      [-1.3071, -1.8390, -0.8650, -1.0303],
                      [ 0.7796,  

In [28]:
%timeit Batch.cat(batches)

168 µs ± 3.21 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

556 µs ± 15.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
